# COX Multiple Regression Tutorial

A comprehensive step-by-step tutorial on performing Cox Proportional Hazards Multiple Regression for survival analysis.

## 📋 Overview

This tutorial will guide you through:
1. Loading and exploring survival data
2. Data preprocessing and preparation
3. Fitting Cox Proportional Hazards models
4. Interpreting results and hazard ratios
5. Visualizing survival curves
6. Checking model assumptions

## 🔧 Setup

First, let's install the required packages.


In [ ]:
# Install required packages
!pip install -q pandas numpy lifelines matplotlib seaborn scipy scikit-learn

print("✅ All packages installed successfully!")


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter, KaplanMeierFitter
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")


## 📊 Step 1: Load and Explore the Data

### Option A: Load from GitHub (if repository is public)

If this notebook is linked to a GitHub repository, you can load data directly:


In [ ]:
# Option A: Load data from GitHub (uncomment to use):
"""
github_url = "https://raw.githubusercontent.com/aralap/3DG_Tutorials/main/data/sample_survival_data.csv"
data = pd.read_csv(github_url)
metadata = pd.read_csv("https://raw.githubusercontent.com/aralap/3DG_Tutorials/main/data/sample_metadata.csv")
"""


### Option B: Generate Sample Data

Alternatively, we can generate sample data directly in the notebook:


In [ ]:
# Option B: Generate sample data
def generate_sample_survival_data(n_samples=500, random_seed=42):
    """Generate synthetic survival data for COX regression analysis."""
    np.random.seed(random_seed)
    
    patient_ids = range(1, n_samples + 1)
    age = np.random.randint(30, 81, n_samples)
    gender = np.random.choice(['Male', 'Female'], n_samples, p=[0.55, 0.45])
    treatment = np.random.choice(['A', 'B', 'C'], n_samples, p=[0.35, 0.35, 0.30])
    biomarker1 = np.random.normal(50, 15, n_samples)
    biomarker2 = np.random.normal(100, 25, n_samples)
    
    # Generate survival times based on covariates
    # Stronger effects for better demonstration:
    # - Treatment B: Strong protective effect (HR ~0.4) - best survival
    # - Treatment A: Moderate protective effect (HR ~0.7) - better survival  
    # - Treatment C: Reference group (HR = 1.0) - control group
    baseline_hazard = 0.015
    treatment_effect = np.where(treatment == 'B', 0.35, 
                                np.where(treatment == 'A', 0.65, 1.0))
    
    # Age effect: Strong positive association (older age = worse survival)
    # Each 10 years increases hazard by ~40%
    age_effect = 1 + (age - 50) / 50 * 1.2  # Normalized around age 50
    
    # Biomarker1 effect: Strong negative association (higher = better survival)
    # Higher biomarker1 values are protective
    biomarker_effect = 1 - (biomarker1 - 50) / 100 * 0.8  # Normalized around 50
    biomarker_effect = np.maximum(0.3, biomarker_effect)  # Prevent negative values
    
    # Combine all effects multiplicatively
    hazard_rate = baseline_hazard * treatment_effect * age_effect * biomarker_effect
    survival_time = np.random.exponential(1 / hazard_rate)
    survival_time = np.minimum(survival_time, 1825)
    
    # Generate censoring (target ~30% censoring rate)
    censoring_time = np.random.uniform(365, 1825, n_samples)
    
    # Determine events and censoring with proper distribution
    # Use a probability-based approach to ensure we get both events and censoring
    random_censor = np.random.random(n_samples)
    censor_mask = random_censor < 0.3  # 30% chance of censoring
    
    # For censored cases, use censoring time
    # For event cases, use survival time
    event = np.where(censor_mask, 0, 1).astype(int)
    survival_time_observed = np.where(
        censor_mask,
        np.minimum(censoring_time, survival_time + np.random.uniform(1, 1825 - survival_time, n_samples)),
        survival_time
    )
    
    data = pd.DataFrame({
        'patient_id': patient_ids,
        'survival_time': survival_time_observed,
        'event': event,
        'age': age,
        'gender': gender,
        'treatment': treatment,
        'biomarker1': np.round(biomarker1, 2),
        'biomarker2': np.round(biomarker2, 2)
    })
    
    return data

# Generate data
data = generate_sample_survival_data(n_samples=500, random_seed=42)
print("✅ Sample data generated successfully!")
print(f"Data shape: {data.shape}")
print(f"Events: {data['event'].sum()}/{len(data)} ({100*data['event'].sum()/len(data):.1f}%)")


In [ ]:
# Display first few rows
print("First 5 rows:")
print(data.head())
print("\n" + "="*80 + "\n")

# Display data info
print("Data information:")
print(data.info())
print("\n" + "="*80 + "\n")

# Display summary statistics
print("Summary statistics:")
print(data.describe())
print("\n" + "="*80 + "\n")

# Check for missing values
print("Missing values:")
print(data.isnull().sum())


### Visualize Data Distribution

Let's create some exploratory visualizations:


In [ ]:
# Create subplots for exploratory analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Age distribution
axes[0, 0].hist(data['age'], bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Age Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Age (years)')
axes[0, 0].set_ylabel('Frequency')

# Survival time distribution
axes[0, 1].hist(data['survival_time'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 1].set_title('Survival Time Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Time (days)')
axes[0, 1].set_ylabel('Frequency')

# Event status
event_counts = data['event'].value_counts().sort_index()
# Handle cases where 0 or 1 might not exist
censored_count = event_counts.get(0, 0)
event_count = event_counts.get(1, 0)
axes[0, 2].bar(['Censored', 'Event'], [censored_count, event_count], 
               color=['orange', 'red'], edgecolor='black')
axes[0, 2].set_title('Event Status Distribution', fontweight='bold')
axes[0, 2].set_ylabel('Count')

# Gender distribution
gender_counts = data['gender'].value_counts()
axes[1, 0].bar(gender_counts.index, gender_counts.values, edgecolor='black')
axes[1, 0].set_title('Gender Distribution', fontweight='bold')
axes[1, 0].set_ylabel('Count')

# Treatment distribution
treatment_counts = data['treatment'].value_counts()
axes[1, 1].bar(treatment_counts.index, treatment_counts.values, edgecolor='black')
axes[1, 1].set_title('Treatment Distribution', fontweight='bold')
axes[1, 1].set_ylabel('Count')

# Biomarker1 distribution
axes[1, 2].hist(data['biomarker1'], bins=20, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1, 2].set_title('Biomarker1 Distribution', fontweight='bold')
axes[1, 2].set_xlabel('Biomarker1 Level')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


## 🔄 Step 2: Data Preprocessing

**Why is preprocessing important for Cox regression?**

Cox regression requires clean, properly formatted data. Preprocessing ensures:
- **No missing values**: The model cannot handle missing data in covariates
- **Proper data types**: Categorical variables must be encoded numerically
- **Correct variable structure**: Duration and event columns must be in the right format

**Key preprocessing steps:**

1. **Handle missing values**
   - Missing data can bias results or cause errors
   - Options: Remove observations (complete case analysis), impute missing values
   - In this tutorial, we'll use complete case analysis (remove rows with any missing values)
   
2. **Encode categorical variables**
   - Cox regression requires numerical inputs
   - Categorical variables (like "Male"/"Female" or "Treatment A/B/C") must be converted to numbers
   - We use LabelEncoder to assign numeric codes (e.g., Male=0, Female=1)
   - Note: This creates ordinal encoding, but for Cox regression this is acceptable as we can interpret coefficients relative to the reference category
   
3. **Prepare the data in the correct format**
   - Cox regression needs: duration column (time), event column (binary), and covariates
   - The lifelines library expects specific column names: 'duration' and 'event'

Let's perform these steps:


In [ ]:
# Check for missing values
print("Missing values before preprocessing:")
print(data.isnull().sum())
print("\n" + "="*80 + "\n")

# Handle missing values (if any)
data_clean = data.dropna().copy()
print(f"Data shape: {data.shape} → {data_clean.shape} after removing missing values")

# Encode categorical variables
le_gender = LabelEncoder()
le_treatment = LabelEncoder()

data_clean['gender_encoded'] = le_gender.fit_transform(data_clean['gender'])
data_clean['treatment_encoded'] = le_treatment.fit_transform(data_clean['treatment'])

print("\nCategorical encoding:")
print(f"Gender: {dict(zip(le_gender.classes_, le_gender.transform(le_gender.classes_)))}")
print(f"Treatment: {dict(zip(le_treatment.classes_, le_treatment.transform(le_treatment.classes_)))}")


### Prepare Data for Cox Regression

**Data Structure Requirements:**

The Cox model requires specific columns with specific meanings:

- **Duration column** (`duration`): 
  - Time until event or censoring
  - Must be continuous (can be days, months, years, etc.)
  - Should be positive (> 0)
  - Represents the time each subject was observed
  
- **Event column** (`event`):
  - Binary indicator of whether the event occurred
  - **1** = Event occurred (e.g., death, disease progression)
  - **0** = Censored (e.g., lost to follow-up, study ended, withdrew)
  - Critical distinction: Event=0 doesn't mean the event won't happen, just that we didn't observe it
  
- **Covariates** (predictor variables):
  - Can be continuous (age, biomarker levels) or categorical (treatment, gender)
  - All must be numerical (categorical variables must be encoded)
  - These are the factors we want to test for association with survival

**Why this structure matters:**

The Cox model uses the duration and event columns to:
1. Rank observations by time (earlier events contribute more information)
2. Calculate hazard rates at each event time
3. Compare covariate values between those who experienced events vs. those at risk

**Important Notes:**
- Duration should represent the actual observed time, not just time until censoring
- Both events and censored observations contribute information (censored observations are in the risk set until their censoring time)
- Missing values in covariates will cause the model to fail - must be handled beforehand


In [ ]:
# Prepare data for Cox regression
cox_data = data_clean[['survival_time', 'event', 'age', 'gender_encoded', 
                       'treatment_encoded', 'biomarker1', 'biomarker2']].copy()

# Rename columns for clarity (lifelines uses 'duration' and 'event')
cox_data.columns = ['duration', 'event', 'age', 'gender', 
                    'treatment', 'biomarker1', 'biomarker2']

print("Data prepared for Cox regression:")
print(cox_data.head())
print(f"\nData shape: {cox_data.shape}")
print(f"Events: {cox_data['event'].sum()}/{len(cox_data)} ({100*cox_data['event'].sum()/len(cox_data):.1f}%)")


## 📈 Step 3: Fit Cox Proportional Hazards Model

**What is Cox Proportional Hazards Regression?**

The Cox Proportional Hazards model (also called Cox regression or Cox PH model) is a semi-parametric survival analysis method that:
- Models the relationship between multiple covariates and survival time
- Estimates hazard ratios (relative risks) for each covariate
- Makes no assumptions about the shape of the baseline hazard function
- Only assumes that hazard ratios are proportional (constant over time)

**Mathematical Foundation:**

The Cox model assumes the hazard function h(t|X) takes the form:

**h(t|X) = h₀(t) × exp(β₁X₁ + β₂X₂ + ... + βₚXₚ)**

Where:
- **h(t|X)**: Hazard at time t given covariates X
- **h₀(t)**: Baseline hazard function (unknown, not estimated)
- **β₁, β₂, ...**: Regression coefficients (what we estimate)
- **X₁, X₂, ...**: Covariate values
- **exp(βᵢXᵢ)**: Hazard ratio contribution for covariate i

**Key Advantages:**
- Can handle multiple covariates simultaneously
- Accounts for censoring automatically
- Does not require specifying the baseline hazard distribution
- Hazard ratios are easily interpretable
- Handles time-to-event data naturally

**What the Model Does:**

1. **Estimates coefficients (β)**: Determines how much each covariate affects the hazard
2. **Calculates hazard ratios**: HR = exp(β) for each covariate
3. **Tests significance**: Determines if each covariate has a statistically significant effect
4. **Provides confidence intervals**: Quantifies uncertainty in estimates

**Model Fitting Process:**

The Cox model uses **partial likelihood** (not full likelihood) because it doesn't estimate the baseline hazard:
- Compares individuals who experience events to those at risk at each event time
- Maximizes the likelihood of observing the event order given the covariate values
- This makes it robust and computationally efficient

Now we'll fit the model with multiple covariates:


In [ ]:
# Initialize Cox Proportional Hazards Fitter
cph = CoxPHFitter()

# Fit the model
print("Fitting Cox Proportional Hazards model...")
cph.fit(cox_data, duration_col='duration', event_col='event')

print("\n✅ Model fitted successfully!")


### View Model Summary

The summary includes:
- **Hazard Ratios (HR)**: Effect of each covariate on survival
- **Confidence Intervals**: 95% CI for each HR
- **P-values**: Statistical significance
- **Concordance Index**: Model performance (similar to AUC)


In [ ]:
# Print detailed summary
print("="*80)
print("COX REGRESSION RESULTS")
print("="*80)
cph.print_summary()
print("="*80)


### Access Model Parameters

You can also access specific model parameters programmatically:


In [ ]:
# Get hazard ratios
print("Hazard Ratios:")
print(cph.hazard_ratios_)
print("\n" + "="*80 + "\n")

# Get confidence intervals
print("95% Confidence Intervals:")
print(cph.confidence_intervals_)
print("\n" + "="*80 + "\n")

# Get p-values
print("P-values:")
print(cph.summary['p'])
print("\n" + "="*80 + "\n")

# Get concordance index (C-index)
print(f"Concordance Index (C-index): {cph.concordance_index_:.4f}")
print("(C-index > 0.7 indicates good discrimination ability)")


## 📊 Step 4: Visualize Results

### Plot Hazard Ratios

**What are Hazard Ratios?**

Hazard ratios (HR) quantify the relationship between covariates and survival. They represent the ratio of hazard rates (instantaneous risk of the event occurring) between different groups or at different covariate levels.

**How to Interpret Hazard Ratio Plots:**

The hazard ratio plot shows each covariate as a point estimate with a horizontal line representing the 95% confidence interval:

- **HR = 1** (vertical reference line): No effect on survival. The covariate has no association with the event.
  
- **HR > 1**: **Increased hazard** (worse survival). The covariate is associated with higher risk of the event.
  - Example: HR = 1.5 means the hazard is 50% higher (or 1.5 times greater) compared to the reference
  - If CI excludes 1, the effect is statistically significant
  
- **HR < 1**: **Decreased hazard** (better survival). The covariate is associated with lower risk of the event.
  - Example: HR = 0.6 means the hazard is 40% lower (or 0.6 times the reference hazard)
  - If CI excludes 1, the effect is statistically significant

**Reading the Plot:**
- Points to the left of HR=1: Protective factors (reduce risk)
- Points to the right of HR=1: Risk factors (increase risk)
- If the confidence interval crosses HR=1: Effect is not statistically significant
- If the confidence interval does not include HR=1: Effect is statistically significant (p < 0.05 typically)

**Real-world Example:**
If "treatment" shows HR = 0.65 with CI [0.50, 0.84]:
- Patients receiving this treatment have 35% lower hazard than the reference group
- The 95% CI [0.50, 0.84] does not include 1, so this is statistically significant
- We can be 95% confident the true HR lies between 0.50 and 0.84


In [ ]:
# Plot hazard ratios with confidence intervals
fig, ax = plt.subplots(figsize=(10, 6))
cph.plot(ax=ax)
ax.set_title('Hazard Ratios with 95% Confidence Intervals', fontsize=14, fontweight='bold')
ax.axvline(x=1, color='red', linestyle='--', alpha=0.5, label='No effect (HR=1)')
ax.set_xlabel('Hazard Ratio', fontsize=12)
ax.set_ylabel('Covariate', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### Kaplan-Meier Survival Curves

**What are Kaplan-Meier Curves?**

Kaplan-Meier (KM) survival curves are non-parametric estimates of the survival function that show the probability of survival over time. They are the most common way to visualize survival data and are particularly useful for comparing survival between groups.

**Key Concepts:**

1. **Survival Function S(t)**: The probability that an individual survives beyond time t
   - S(0) = 1 (100% survival at baseline)
   - S(t) decreases (or stays constant) over time
   - Typically ranges from 1.0 (100% survival) to 0.0 (0% survival)

2. **Censoring**: The KM estimator handles censored observations by removing them from the risk set at the time of censoring, but not treating them as events.

3. **Step Function**: KM curves are step functions that drop at each event time. Between events, the curve is flat (horizontal), indicating constant survival probability.

**How to Interpret Kaplan-Meier Curves:**

- **Y-axis**: Survival probability (0 to 1 or 0% to 100%)
- **X-axis**: Time (in our case, days since study entry)
- **Steeper drops**: Indicate more events occurring in that time period (worse survival)
- **Flatter curves**: Indicate fewer events (better survival)
- **Vertical lines (ticks)**: Often mark censoring times (patients who were censored)

**Comparing Groups:**

When comparing survival curves between groups (e.g., different treatments):
- **Curve stays higher**: Better survival (fewer events over time)
- **Curve drops faster**: Worse survival (more events over time)
- **Separation between curves**: Suggests a difference between groups
- **Curves that cross**: May indicate non-proportional hazards (violates Cox model assumption)

**In Our Plot:**

We're comparing survival by:
1. **Gender**: Do males and females have different survival?
2. **Treatment**: Do different treatment groups have different survival?

Look for:
- Which group has the highest curve at the end (best survival)
- Whether curves separate early or late (when differences appear)
- Whether curves remain parallel (proportional hazards) or cross (non-proportional)


In [ ]:
# Create Kaplan-Meier curves
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# By gender
kmf = KaplanMeierFitter()
ax1 = axes[0]
for gender in data_clean['gender'].unique():
    mask = data_clean['gender'] == gender
    kmf.fit(data_clean[mask]['survival_time'], 
            data_clean[mask]['event'], 
            label=f'{gender}')
    kmf.plot_survival_function(ax=ax1)

ax1.set_title('Kaplan-Meier Curves by Gender', fontsize=12, fontweight='bold')
ax1.set_xlabel('Time (days)', fontsize=11)
ax1.set_ylabel('Survival Probability', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.legend()

# By treatment
kmf = KaplanMeierFitter()
ax2 = axes[1]
for treatment in sorted(data_clean['treatment'].unique()):
    mask = data_clean['treatment'] == treatment
    kmf.fit(data_clean[mask]['survival_time'], 
            data_clean[mask]['event'], 
            label=f'Treatment {treatment}')
    kmf.plot_survival_function(ax=ax2)

ax2.set_title('Kaplan-Meier Curves by Treatment', fontsize=12, fontweight='bold')
ax2.set_xlabel('Time (days)', fontsize=11)
ax2.set_ylabel('Survival Probability', fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()


## ✅ Step 5: Check Model Assumptions

### Proportional Hazards Assumption

**What is the Proportional Hazards Assumption?**

The Cox Proportional Hazards model assumes that the hazard ratio between any two groups remains **constant over time**. This means that if Treatment A has twice the hazard of Treatment B at the beginning of the study, it should have twice the hazard throughout the entire follow-up period.

**Why is this important?**

If the proportional hazards assumption is violated:
- Hazard ratios can be misleading (may average out time-varying effects)
- The model may not accurately represent the data
- Alternative approaches may be needed (stratified Cox models, time-dependent covariates, etc.)

**How do we test this? Using Schoenfeld Residuals**

Schoenfeld residuals are a diagnostic tool specifically designed to test the proportional hazards assumption. Here's what they are:

1. **What are Schoenfeld Residuals?**
   - Calculated for each covariate at each event time
   - Measure the difference between the observed covariate value and the expected value given the model
   - If hazards are proportional, residuals should be randomly distributed around zero over time
   - If hazards are non-proportional, residuals will show a systematic pattern (trend) over time

2. **How are they calculated?**
   - At each event time, compute the weighted average of covariate values for all individuals at risk
   - The residual = observed covariate value - expected (weighted average) value
   - Only calculated for individuals who experienced events (not for censored observations)

3. **Interpreting Schoenfeld Residual Plots:**
   - **Horizontal line around zero**: Proportional hazards assumption satisfied ✅
   - **Trend or slope over time**: Violation of proportional hazards ❌
   - **Curved patterns**: May indicate time-dependent effects
   - **Statistical test**: The `check_assumptions` function performs a statistical test (typically Grambsch-Therneau test) that tests whether there's a correlation between residuals and time
     - **p > 0.05**: Proportional hazards assumption is reasonable
     - **p < 0.05**: Evidence against proportional hazards (assumption violated)

4. **What if the assumption is violated?**
   - Use **stratified Cox models**: Allow baseline hazard to differ between groups
   - Include **time-dependent covariates**: Model how effects change over time (e.g., `treatment × time`)
   - Consider **parametric survival models**: Weibull, exponential, etc. that don't require proportional hazards
   - Use **piecewise Cox models**: Different hazards in different time periods

**Example Interpretation:**

If we see a positive slope in Schoenfeld residuals for "age":
- This suggests that the effect of age on survival increases over time
- Older patients may have worse outcomes that become more pronounced as time progresses
- We might need to include an interaction term like `age × time` or use a stratified model

Let's run the diagnostic test:


In [ ]:
# Check proportional hazards assumption
print("Checking proportional hazards assumption...")
print("="*80)

try:
    cph.check_assumptions(cox_data, p_value_threshold=0.05, show_plots=True)
    print("\n✅ Proportional hazards assumption checked.")
except Exception as e:
    print(f"\n⚠️  Note: {e}")
    print("This is normal if the test requires specific conditions.")
    print("Visual inspection of Schoenfeld residuals can still be performed.")


## 📝 Step 6: Interpreting Results

### How to Interpret Hazard Ratios

**Understanding Hazard Ratios (HR) in Detail:**

1. **HR = 1.0**: The covariate has **no effect** on survival
   - The hazard for individuals with this covariate value equals the reference group
   - Example: If treatment HR = 1.0, the treatment has no effect compared to control

2. **HR > 1.0**: Associated with **worse survival** (increased hazard)
   - The risk of event is higher for this group/value compared to the reference
   - **Interpretation formula**: For HR > 1, the hazard is (HR - 1) × 100% higher
   - Example: HR = 1.5 means hazard is 50% higher (or 1.5 times greater)
   - Example: HR = 2.0 means hazard is 100% higher (double the risk, or 2× greater)
   - For **continuous variables**: HR represents the hazard for each 1-unit increase
     - Example: If age HR = 1.03, each additional year increases hazard by 3%

3. **HR < 1.0**: Associated with **better survival** (decreased hazard)
   - The risk of event is lower for this group/value compared to the reference
   - **Interpretation formula**: For HR < 1, the hazard is (1 - HR) × 100% lower
   - Example: HR = 0.7 means hazard is 30% lower (or 0.7 times the reference)
   - Example: HR = 0.5 means hazard is 50% lower (half the risk)

### Comprehensive Example Interpretation

**Scenario**: Treatment B has HR = 0.65 with 95% CI [0.45, 0.94] and p = 0.02

**Step-by-step interpretation:**

1. **Effect Size**: HR = 0.65 means patients receiving Treatment B have 65% of the hazard compared to the reference group (Treatment C)
   - This translates to: Treatment B reduces the hazard by **35%** (1 - 0.65 = 0.35)

2. **Statistical Significance**: p = 0.02 < 0.05
   - We reject the null hypothesis (HR = 1)
   - There is statistically significant evidence that Treatment B differs from the reference

3. **Confidence Interval**: 95% CI [0.45, 0.94]
   - We are 95% confident the true hazard ratio lies between 0.45 and 0.94
   - The CI does **not** include 1.0, confirming statistical significance
   - The CI is relatively wide, suggesting some uncertainty in the estimate (may need larger sample size)

4. **Clinical Interpretation**:
   - Treatment B is **protective** (reduces risk of event)
   - On average, patients on Treatment B have 35% lower risk of experiencing the event
   - This is a **clinically meaningful** effect if it represents a reduction in death, disease progression, etc.

### Interpreting Continuous Variables

**Example**: Age has HR = 1.02 with 95% CI [1.01, 1.03], p < 0.001

- HR = 1.02 means each additional **year of age** increases hazard by 2%
- This is a **small but significant** effect
- For a 10-year age difference: HR = 1.02^10 ≈ 1.22 (22% increase in hazard)
- **Clinical interpretation**: Older patients have worse survival, with each year adding 2% to the hazard

### Model Performance Metrics

- **Concordance Index (C-index)**:
  - Measures how well the model ranks patients by their survival time
  - Interpretation similar to AUC in logistic regression:
    - **0.5**: Random guessing (no discrimination) - like flipping a coin
    - **0.6-0.7**: Poor to fair discrimination
    - **0.7-0.8**: Acceptable discrimination - model is useful
    - **0.8-0.9**: Excellent discrimination - model is very good
    - **>0.9**: Outstanding discrimination - model is excellent
  - In survival analysis, C-index > 0.7 is generally considered acceptable

- **Log-likelihood**:
  - Measures how well the model fits the data
  - **Higher values = better fit** (less negative is better)
  - Used to compare nested models (e.g., full model vs. reduced model)
  - Not directly interpretable, but useful for model comparison

- **AIC (Akaike Information Criterion)**:
  - Balances model fit with model complexity
  - **Lower values = better model** (penalizes for too many parameters)
  - Useful for selecting between non-nested models
  - Rule of thumb: Difference of >2 AIC units suggests one model is better

### Reading the Full Summary Table

When you look at `cph.print_summary()`, you'll see:
- **coef**: Log-hazard ratio (not directly interpretable, but used to calculate HR)
- **exp(coef)**: Hazard ratio (the interpretable value)
- **se(coef)**: Standard error of the log-hazard ratio
- **coef lower 95%**: Lower bound of 95% CI for log-hazard ratio
- **coef upper 95%**: Upper bound of 95% CI for log-hazard ratio
- **exp(coef) lower 95%**: Lower bound of 95% CI for hazard ratio
- **exp(coef) upper 95%**: Upper bound of 95% CI for hazard ratio
- **z**: Z-statistic (coef / se(coef))
- **p**: P-value for testing HR = 1
- **-log2(p)**: Negative log-base-2 of p-value (larger = more significant)


## 🎯 Additional Analyses

### Predict Individual Survival Probabilities

You can use the fitted model to predict survival probabilities for new patients:


In [ ]:
# Example: Predict survival probability for a new patient
new_patient = pd.DataFrame({
    'age': [65],
    'gender': [1],  # Encoded: 1 = Female, 0 = Male
    'treatment': [1],  # Encoded: 1 = Treatment B, 0 = Treatment A, 2 = Treatment C
    'biomarker1': [55.0],
    'biomarker2': [110.0]
})

# Predict survival probability at specific time points
time_points = [365, 730, 1095, 1460]  # 1, 2, 3, 4 years
survival_probs = cph.predict_survival_function(new_patient, times=time_points)

print("Survival Probabilities for New Patient:")
print(f"Age: {new_patient['age'].iloc[0]}, Gender: Female, Treatment: B")
print(f"Biomarker1: {new_patient['biomarker1'].iloc[0]}, Biomarker2: {new_patient['biomarker2'].iloc[0]}")
print("\nTime (days) | Survival Probability")
print("-" * 40)
for t, prob in zip(time_points, survival_probs.iloc[0]):
    print(f"{t:>12} | {prob:.4f}")


## 📚 Summary

In this tutorial, we learned how to:

1. ✅ Load and explore survival data
2. ✅ Preprocess data for Cox regression
3. ✅ Fit Cox Proportional Hazards models with multiple covariates
4. ✅ Interpret hazard ratios and model results
5. ✅ Visualize results with Kaplan-Meier curves
6. ✅ Check proportional hazards assumption
7. ✅ Make predictions for new patients

## 🔗 Next Steps

- Try different combinations of covariates
- Explore interaction terms (e.g., treatment × age)
- Consider stratified models if proportional hazards are violated
- Compare models using AIC or likelihood ratio tests

## 📖 Resources

- [Lifelines Documentation](https://lifelines.readthedocs.io/)
- [Survival Analysis Guide](https://www.statsdirect.com/help/survival_analysis/cox_regression.htm)
- Cox, D. R. (1972). Regression models and life-tables. *Journal of the Royal Statistical Society: Series B*, 34(2), 187-202.

---

**Happy Analyzing! 📊**
